<a href="https://colab.research.google.com/github/pandov/cups-mail/blob/master/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
from src.nn import BACTERIA, SupervisedRunner, get_segmentation_components
logdir = './logs/segmentation'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
%%capture
dataset = BACTERIA(target='mask')
for i, loaders in enumerate(dataset.crossval(kfold=4, batch_size=16)):
    model, optimizer, scheduler, criterion, callbacks = get_segmentation_components(1, 1, 1)
    model.train()
    runner = SupervisedRunner()
    runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        callbacks=callbacks,
        loaders=loaders,
        logdir=f'{logdir}/{i}',
        num_epochs=100,
        # verbose=True,
    )

In [ ]:
! zip -r {logdir}.zip {logdir}
! git pull origin master
! git add logs
! git commit -m 'Changed from Colab'
! git push -u origin master